In [ ]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import keras_tuner
import tensorflow_datasets as tfds
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Loading train, validation and test sets

In [ ]:
ds = tfds.load('malaria', 
               split=('train[:60%]', 'train[60%:80%]', 'train[80%:]'), 
               shuffle_files=True,
              data_dir='/content/gdrive/MyDrive/datasets/Malaria',
               batch_size=32,
              download=True,
              as_supervised=True,
              with_info=False)
ds

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

# Model optimization

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(None, None, 3))
    x = keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    init_filters = hp.Choice('init_filters', [16, 32, 64]) #No. filters in the 1st convol layer
    kernel_size = hp.Choice('kernel_size', [2, 3])
    num_chan = init_filters
    x = keras.layers.Conv2D(
        filters=init_filters,
        kernel_size=kernel_size,
        strides=(1, 1),
        padding="valid",
        activation='relu')(x)
    for i in range(hp.Int("num_convols", 3, 10, 1)):
        # i is the number of convlutions with MaxPooling
        # There is another convolution with GlobalMaxPooling
        if i != 0:
            x = keras.layers.MaxPooling2D(
                pool_size=(2, 2), 
                strides=None, 
                padding="valid")(x)
            num_chan = init_filters * (i + 1)
            x = keras.layers.Conv2D(
                filters=num_chan,
                kernel_size=kernel_size,
                strides=(1, 1),
                padding="valid",
                activation='relu')(x)
    x = keras.layers.GlobalMaxPooling2D()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=num_chan, #Number of channels in the last layer
                            activation="relu",
                          kernel_regularizer=None)(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='Adam',
         loss='binary_crossentropy',
         metrics=['accuracy'])    
    return model

In [ ]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=25,
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=8)

In [ ]:
tuner.search(ds[0], epochs=15, validation_data=ds[1])

Trial 9 Complete [00h 10m 54s]
val_accuracy: 0.9637089371681213

Best val_accuracy So Far: 0.9637089371681213
Total elapsed time: 00h 56m 23s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
32                |64                |init_filters
2                 |2                 |kernel_size
7                 |4                 |num_convols



/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Epoch 1/15


In [ ]:
tuner.results_summary(1)

NameError: ignored

# The program crashes for unknown reasons, apparently when the model is too big: initial number of filters at 128 (crashes on start) or the number of weights is large due to a combination of parameters 32 filters and 7 convolutions.
# I am going to try manual optimization.